# Regions V3 (Fix wrong regions) - Living Income Benchmark V4 - CPI V4

Here we will map the li_benchmark_data.csv and cpi_data.csv only

In [1]:
import pandas as pd
import re

In [2]:
def remove_extra_whitespace(val):
    temp = val.split()
    return ' '.join(temp)

## Load files

In [3]:
countries = pd.read_csv("./countries.csv")

In [4]:
countries.tail()

,id,country,currency,abbreviation
205,206,Martinique,Euro,EUR
206,207,Montserrat,East Caribbean Dollar,XCD
207,208,"San Marino, Rep. Of",Euro,EUR
208,209,"São Tomé and Príncipe, Dem. Rep. of",Dobra,STN
209,210,Syrian Arab Rep.,Syrian Pound,SYP


In [5]:
lib = pd.read_csv("./li_benchmark_data.csv")

In [6]:
lib.tail()

,country,region,source,household_size,nr_adults,year,LCU,USD,EUR,household_equiv,links
279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
lib = lib.dropna(how="all")

In [8]:
lib["id"] = lib.reset_index().index + 1

In [9]:
lib.columns

Index(['country', 'region', 'source', 'household_size', 'nr_adults', 'year',
       'LCU', 'USD', 'EUR', 'household_equiv', 'links', 'id'],
      dtype='object')

In [10]:
lib[lib['country'] == "Kenya"]

,country,region,source,household_size,nr_adults,year,LCU,USD,EUR,household_equiv,links,id
59,Kenya,Rural / Kericho,Calculated from Global Living Wage Coalition,5.0,2.0,2023.0,570492.0,NaN,NaN,2.4,https://globallivingwage.org/living-wage-bench...,60
60,Kenya,Non-metropolitan Urban / Lake Naivasha,Calculated from Global Living Wage Coalition,5.0,2.0,2023.0,627012.0,NaN,NaN,2.4,https://globallivingwage.org/living-wage-bench...,61


## Regions V4
- https://app.asana.com/0/1205513346619877/1207662929082480/f
---
Issue:
- The region must not split by their "/"
- The name of region e.g. "Rural / All" that will be the exact region name
- And for Vietnam, there's still a region call Hong Min City that must not be there on the regions


Solutions:
- check the region data and make adjusment on the seeder, including
- check the benchmark data seeder (try if we can do truncate)
- make sure we doesn't truncate the region data, because the region linked into segment

In [11]:
def find_country(val):
    # manual match
    if val == "Turkey":
        val = "Turkiye"
    # if val == "Anguilla":
    #     print("404", val)
    #     return None
    if val == "Central African Rep.":
        val = "Central African Republic"
    if val == "Congo, Dem. Rep. of the":
        val = "Congo, Dem. Rep."
    if val == "Congo, Rep. of":
        val = "Congo, Rep."
    if val == "Côte d'Ivoire":
        val = "Cote d'Ivoire"
    if val == "Curaçao, Kingdom of the Netherlands":
        val = "Curacao"
    if val == "Czech Rep.":
        val = "Czech Republic"
    if val == "Dominican Rep.":
        val = "Dominican Republic"
    if val == "Egypt, Arab Rep. of":
        val = "Egypt, Arab Rep."
    # if val == "Guadeloupe":
    #     print("404", val)
    #     return None
    if val == "Iran, Islamic Rep. of":
        val = "Iran, Islamic Rep."
    if val == "Korea, Rep. of":
        val = "Korea, Rep."
    if val == "Kyrgyz Rep.":
        val = "Kyrgyz Republic"
    if val == "Lao People's Dem. Rep.":
        val = "Lao PDR"
    # if val == "Martinique":
    #     print("404", val)
    #     return None
    if val == "Micronesia, Federated States of":
        val = "Micronesia, Fed. Sts."
    # if val == "Montserrat":
    #     print("404", val)
    #     return None
    # if val == "San Marino, Rep. of":
    #     print("404", val)
    #     return None
    # if val == "São Tomé and Príncipe, Dem. Rep. of":
    #     print("404", val)
    #     return None
    if val == "Sint Maarten, Kingdom of the Netherlands":
        val = "Sint Maarten (Dutch part)"
    if val == "Slovak Rep.":
        val = "Slovak Republic"
    # if val == "Syrian Arab Rep.":
    #     print("404", val)
    #     return None
    if val == "Venezuela, Rep. Bolivariana de":
        val = "Venezuela, RB"
    if val == "Yemen, Rep. of":
        val = "Yemen, Rep."
    # EOL manual match
    
    find_country = countries[countries["country"].str.lower() == str(val).lower()]
    if find_country.empty:
        # Another custom way
        val_lower = str(val).lower()
        val_tmp = val_lower.lower().split(',')
        for index, row in countries.iterrows():
            # Convert the country name in the DataFrame to lowercase
            country_name = row['country'].lower()
            if val_lower == 'nan':
                return None
            if country_name not in val_tmp:
                continue
            return row["id"]
        # EOL Another custom way
        print("404", val)
        return val
    return find_country["id"].to_list()[0]

In [12]:
df = pd.read_csv("./li_benchmark_data.csv")

In [13]:
df.columns

Index(['country', 'region', 'source', 'household_size', 'nr_adults', 'year',
       'LCU', 'USD', 'EUR', 'household_equiv', 'links'],
      dtype='object')

In [14]:
region_source = df[["country", "region"]]

In [15]:
region_temp = region_source

In [16]:
region_temp = region_temp[region_temp["region"].notna()]

In [17]:
region_temp['country'] = region_temp['country'].str.replace('\n', ' ').str.strip()
region_temp['region'] = region_temp['region'].str.replace('\n', ' ').str.strip()
region_temp['region'] = region_temp['region'].str.replace('\r', ' ').str.strip()
region_temp['region'] = region_temp['region'].str.replace(r'^\s+|\s+?$', '', regex=True)
region_temp['region'] = region_temp['region'].apply(remove_extra_whitespace)

/var/folders/00/btls1cns2d544mn2rkm3v4v40000gn/T/ipykernel_20467/1365577662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_temp['country'] = region_temp['country'].str.replace('\n', ' ').str.strip()
/var/folders/00/btls1cns2d544mn2rkm3v4v40000gn/T/ipykernel_20467/1365577662.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_temp['region'] = region_temp['region'].str.replace('\n', ' ').str.strip()
/var/folders/00/btls1cns2d544mn2rkm3v4v40000gn/T/ipykernel_20467/1365577662.py:3: SettingWit

In [18]:
region_temp.tail(10)

,country,region
96,Tunisia,Rural / All
97,Turkey,Rural / All
98,Turkey,Urban / All
99,Uganda,Rural / Lake Victoria Basin
100,Vietnam,Urban / Minimum Wage Region 1
101,Vietnam,Urban / Minimum Wage Region 2
102,Vietnam,Urban / Minimum Wage Region 3
103,Vietnam,Rural / Minimum Wage Region 4
104,Zambia,Rural / All
105,Zimbabwe,Rural / All


### Map regions with countries

In [19]:
region_temp["country_id"] = region_temp["country"].apply(find_country)

/var/folders/00/btls1cns2d544mn2rkm3v4v40000gn/T/ipykernel_20467/2572481390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_temp["country_id"] = region_temp["country"].apply(find_country)


In [20]:
region_temp.tail(10)

,country,region,country_id
96,Tunisia,Rural / All,185
97,Turkey,Rural / All,186
98,Turkey,Urban / All,186
99,Uganda,Rural / Lake Victoria Basin,190
100,Vietnam,Urban / Minimum Wage Region 1,199
101,Vietnam,Urban / Minimum Wage Region 2,199
102,Vietnam,Urban / Minimum Wage Region 3,199
103,Vietnam,Rural / Minimum Wage Region 4,199
104,Zambia,Rural / All,202
105,Zimbabwe,Rural / All,203


### Group region with same name

In [21]:
group_df = region_temp.groupby('region').agg({
    'country': lambda x: list(x),
    'country_id': lambda x: list(x)
}).reset_index()

In [22]:
group_df = group_df[group_df['region'].str.strip() != '']

In [23]:
group_df['region'] = group_df['region'].str.replace(r'^\s+|\s+?$', '', regex=True)

In [24]:
group_df['country_id'] = group_df['country_id'].apply(list)
group_df['country'] = group_df['country'].apply(list)

In [25]:
group_df["id"] = group_df.reset_index().index + 1

In [26]:
new_column_order = ["id", "region", "country_id", "country"]

In [27]:
regions = group_df[new_column_order]

### Export Region to CSV

In [28]:
regions[regions['region'].str.contains("coffee-producing households in Cuanza Sul province")]

,id,region,country_id,country
40,41,Rural / coffee-producing households in Cuanza ...,[5],[Angola]


In [29]:
regions.tail(5)

,id,region,country_id,country
67,68,"Urban / Tiruppur, Tamil Nadu",[84],[India]
68,69,Urban / Urban Guatemala,[75],[Guatemala]
69,70,Urban / Zhengzhou,[40],[China]
70,71,Urban-Rural / Caribbean coast,[41],[Colombia]
71,72,Yucatán / Rural Areas and Small Towns in Yucatan,[119],[Mexico]


In [30]:
regions.to_csv('./regions.csv', index=False)

## Living Income Benchmark V4

### Get country id

In [31]:
countries.columns

Index(['id', 'country', 'currency', 'abbreviation'], dtype='object')

In [32]:
def find_new_country(val):
    if val == "Turkey":
        val = "Turkiye"
    country = countries[countries["country"].str.lower() == str(val).lower()]
    if country.empty:
        print("404", val)
        return None
    return country["id"].to_list()[0]

In [33]:
lib["country_id"] = lib["country"].apply(find_new_country)

In [34]:
lib.tail()

,country,region,source,household_size,nr_adults,year,LCU,USD,EUR,household_equiv,links,id,country_id
101,Vietnam,Urban / Minimum Wage Region 2,Calculated from Global Living Wage Coalition,4.0,2.0,2023.0,1.473626e+08,NaN,NaN,2.1,NaN,102,199
102,Vietnam,Urban / Minimum Wage Region 3,Calculated from Global Living Wage Coalition,4.0,2.0,2023.0,1.431112e+08,NaN,NaN,2.1,NaN,103,199
103,Vietnam,Rural / Minimum Wage Region 4,Calculated from Global Living Wage Coalition,4.0,2.0,2023.0,1.218759e+08,NaN,NaN,2.1,NaN,104,199
104,Zambia,Rural / All,Calculated from Global Living Wage Coalition,6.0,2.0,2023.0,5.410944e+04,NaN,NaN,2.7,NaN,105,202
105,Zimbabwe,Rural / All,Calculated from Global Living Wage Coalition,5.0,2.0,2023.0,4.368000e+03,NaN,NaN,2.4,NaN,106,203


### Get region id

In [35]:
regions.tail()

,id,region,country_id,country
67,68,"Urban / Tiruppur, Tamil Nadu",[84],[India]
68,69,Urban / Urban Guatemala,[75],[Guatemala]
69,70,Urban / Zhengzhou,[40],[China]
70,71,Urban-Rural / Caribbean coast,[41],[Colombia]
71,72,Yucatán / Rural Areas and Small Towns in Yucatan,[119],[Mexico]


In [36]:
regions.columns

Index(['id', 'region', 'country_id', 'country'], dtype='object')

In [37]:
lib['country'] = lib['country'].str.replace('\n', ' ').str.strip()
lib['region'] = lib['region'].str.replace('\n', ' ').str.strip()
lib['region'] = lib['region'].str.replace('\r', ' ').str.strip()
lib['region'] = lib['region'].str.replace(r'^\s+|\s+?$', '', regex=True)
lib['region'] = lib['region'].apply(remove_extra_whitespace)

In [38]:
lib.tail()

,country,region,source,household_size,nr_adults,year,LCU,USD,EUR,household_equiv,links,id,country_id
101,Vietnam,Urban / Minimum Wage Region 2,Calculated from Global Living Wage Coalition,4.0,2.0,2023.0,1.473626e+08,NaN,NaN,2.1,NaN,102,199
102,Vietnam,Urban / Minimum Wage Region 3,Calculated from Global Living Wage Coalition,4.0,2.0,2023.0,1.431112e+08,NaN,NaN,2.1,NaN,103,199
103,Vietnam,Rural / Minimum Wage Region 4,Calculated from Global Living Wage Coalition,4.0,2.0,2023.0,1.218759e+08,NaN,NaN,2.1,NaN,104,199
104,Zambia,Rural / All,Calculated from Global Living Wage Coalition,6.0,2.0,2023.0,5.410944e+04,NaN,NaN,2.7,NaN,105,202
105,Zimbabwe,Rural / All,Calculated from Global Living Wage Coalition,5.0,2.0,2023.0,4.368000e+03,NaN,NaN,2.4,NaN,106,203


In [39]:
def find_region(val):
    region = regions[regions["region"].str.lower() == str(val).lower()]
    if region.empty:
        print("404", val)
        return None
    return region["id"].to_list()[0]

In [40]:
lib["region_id"] = lib["region"].apply(find_region)

### Export Benchmark to CSV

In [41]:
lib = lib.dropna(subset=['region_id']) # remove row when region_id column is None

In [42]:
lib["id"] = lib.reset_index().index + 1

In [43]:
lib.columns

Index(['country', 'region', 'source', 'household_size', 'nr_adults', 'year',
       'LCU', 'USD', 'EUR', 'household_equiv', 'links', 'id', 'country_id',
       'region_id'],
      dtype='object')

In [44]:
lib = lib[['id', 'country_id',
       'region_id', 'country', 'region', 'source', 'household_size', 'nr_adults', 'year',
       'LCU', 'USD', 'EUR', 'household_equiv', 'links']]

In [45]:
lib.tail(5)

,id,country_id,region_id,country,region,source,household_size,nr_adults,year,LCU,USD,EUR,household_equiv,links
101,102,199,60,Vietnam,Urban / Minimum Wage Region 2,Calculated from Global Living Wage Coalition,4.0,2.0,2023.0,1.473626e+08,NaN,NaN,2.1,NaN
102,103,199,61,Vietnam,Urban / Minimum Wage Region 3,Calculated from Global Living Wage Coalition,4.0,2.0,2023.0,1.431112e+08,NaN,NaN,2.1,NaN
103,104,199,33,Vietnam,Rural / Minimum Wage Region 4,Calculated from Global Living Wage Coalition,4.0,2.0,2023.0,1.218759e+08,NaN,NaN,2.1,NaN
104,105,202,17,Zambia,Rural / All,Calculated from Global Living Wage Coalition,6.0,2.0,2023.0,5.410944e+04,NaN,NaN,2.7,NaN
105,106,203,17,Zimbabwe,Rural / All,Calculated from Global Living Wage Coalition,5.0,2.0,2023.0,4.368000e+03,NaN,NaN,2.4,NaN


In [46]:
lib.to_csv("./li_benchmark.csv", index=False)

## CPI V4

In [47]:
cpi_df = pd.read_csv("./cpi_data.csv")

In [48]:
cpi_df = cpi_df.rename(columns={"Country Name": "country"})

In [49]:
cpi_df = cpi_df.dropna(how="all")

In [50]:
cpi_df.head()

,country,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Afghanistan, Islamic Rep. of",97.417222,101.970499,101.295751,105.736448,110.997842,111.692854,114.264439,NaN,NaN,NaN,NaN
1,Albania,89.264316,90.715633,92.435759,93.614714,95.474522,97.410802,98.785357,100.386555,102.435919,109.324942,NaN
2,Algeria,160.100000,164.770000,172.653333,183.699167,193.970000,202.252500,206.200000,211.180000,226.440000,247.420833,270.485833
3,Angola,26.514746,28.445122,31.106440,40.654380,52.787468,63.149087,73.935554,90.402140,113.684573,137.962244,156.785953
4,Anguilla,107.442500,107.170000,106.127500,105.573687,106.940000,107.342500,108.217500,107.705000,109.640000,115.730000,119.672500


### Map with Countries

In [51]:
cpi_df["country_id"] = cpi_df["country"].apply(find_country)

### Transform CPI table

In [52]:
# transform int country, country_id, year, value format
melted_df = pd.melt(cpi_df, id_vars=['country', 'country_id'], var_name='year', value_name='value')
melted_df['value'] = melted_df['value'].fillna(0)

In [53]:
melted_df.reset_index(drop=True, inplace=True)
melted_df["id"] = melted_df.reset_index().index + 1

In [54]:
new_column_order = ["id", "country", "country_id", "year", "value"]

In [55]:
cpi_df = melted_df[new_column_order]

### Export CPI to CSV

In [56]:
cpi = cpi_df.dropna()

In [57]:
cpi = cpi.drop_duplicates(subset=["country_id", "year"])

In [58]:
cpi.head()

,id,country,country_id,year,value
0,1,"Afghanistan, Islamic Rep. of",1,2013,97.417222
1,2,Albania,2,2013,89.264316
2,3,Algeria,3,2013,160.100000
3,4,Angola,5,2013,26.514746
4,5,Anguilla,204,2013,107.442500


In [59]:
cpi.to_csv('cpi.csv', index=False)